In [1]:
import pandas as pd  
import numpy as np

#df=pf.read_csv("")

## Potential sites location data

In [2]:
df_ps=pd.read_csv("/home/mint/Desktop/Agam_Strabag/files/equip_latlon.csv")
df_ps=df_ps[['Equip_LATITUDE','Equip_LONGITUDE']].drop_duplicates()

## Demand Points as per KMeans cluster with LatLon

In [3]:
df_dp=pd.read_csv("/home/mint/Desktop/Agam_Strabag/files/KMeans_clusters.csv")
df_dp.head()

,Cluster,Accident_LATITUDE,Accident_LONGITUDE,Accident_Frequency
0,0,17.354695,78.670910,2
1,1,17.549523,78.277730,5
2,2,17.350922,78.359623,4
3,3,17.595073,78.505587,4
4,4,17.213877,78.534427,5


## Sources 16 (2 private hospitals) Trauma Centre-- 19Toll PLaza

In [4]:
df_s=pd.read_excel("/home/mint/Desktop/Agam_Strabag/files/sources_hosp_toll.xlsx")
df_s

,ID,Source_Latitude,Source_Longitude,ambulances
0,1,17.40713,78.33044,1
1,1,17.46308,78.24922,0
2,1,17.53758,78.23448,1
3,1,17.55849,78.31313,0
4,1,17.59440,78.40774,1
5,1,17.59829,78.49089,0
6,1,17.57643,78.57166,1
7,1,17.44174,78.66665,1
8,2,17.37449,78.68084,0
9,2,17.31975,78.65680,1


## Defining the Demand Points (dp) --Ambulance Source (s) -- No. of Ambulance each location (n) -- Potential Sites (y)

In [7]:
dp=len(df_dp)
s=10 #Places where ambulances are located
n=1 #number of ambulances at each site
y=16  #POtential sites are all the trauma centres

In [8]:
df_s_ambavail=df_s.loc[df_s['ambulances']==1]
df_s_ambnotavail=df_s.loc[df_s['ambulances']==0]
df_s

,ID,Source_Latitude,Source_Longitude,ambulances
0,1,17.40713,78.33044,1
1,1,17.46308,78.24922,0
2,1,17.53758,78.23448,1
3,1,17.55849,78.31313,0
4,1,17.59440,78.40774,1
5,1,17.59829,78.49089,0
6,1,17.57643,78.57166,1
7,1,17.44174,78.66665,1
8,2,17.37449,78.68084,0
9,2,17.31975,78.65680,1


## Plotting the demand points (accident locations) and current sources (ambulance location in trauma centres)

In [20]:
import plotly.express as px


# Function to create scatter plot
def plot_centroids_ambulance(data,df_ambulances,df_trauma ,title):
    df_accidents = pd.DataFrame(data, columns=['Accident_LATITUDE', 'Accident_LONGITUDE', 'Accident_Frequency','Cluster'])
    
    fig = px.scatter_mapbox(df_accidents, lat="Accident_LATITUDE", lon="Accident_LONGITUDE", color="Accident_Frequency", size='Accident_Frequency',
                            color_continuous_scale='Reds', size_max=25, zoom=8.5,
                            mapbox_style="carto-positron", title=title, hover_name='Cluster')
                            #,labels={'Accident_Frequency': 'Sum of Accident Frequency'})

        # Add trace for trauma centers
    trace_trauma = px.scatter_mapbox(df_trauma, lat="Source_Latitude", lon="Source_Longitude",
                                     color_discrete_sequence=['black'], 
                                     size_max=15, zoom=8.5,
                                     mapbox_style="carto-positron").data[0]
    trace_trauma.name = 'Trauma Centers'
    fig.add_trace(trace_trauma)


        # Add trace for ambulances
    trace_ambulances = px.scatter_mapbox(df_ambulances, lat="Source_Latitude", lon="Source_Longitude",
                                         color_discrete_sequence=['blue'], 
                                         size_max=30, zoom=8.5,
                                         mapbox_style="carto-positron").data[0]
    trace_ambulances.name = 'Ambulances'
    fig.add_trace(trace_ambulances)

    fig.update_layout(legend=dict(
    title=title,
    orientation='h',  # horizontal legend
    yanchor='bottom',
    y=1.02,
    xanchor='right',
    x=1
    ))

    fig.show()

plot_centroids_ambulance(df_dp,df_s_ambavail,df_s, 'Fatal Accident Cluster and Ambulances location Yellow--Trauma Centre  Blue--Ambulance deployed')

In [21]:
dp_cords_list=[]
s_cords_list=[]

for _,row in df_dp.iterrows():
    dp_cords_list.append([row['Accident_LATITUDE'],row['Accident_LONGITUDE']])

for _,row in df_s.iterrows():
    s_cords_list.append([row['Source_Latitude'],row['Source_Longitude']])

In [22]:
source_cords=pd.DataFrame({"Source":s_cords_list})
dp_cords=pd.DataFrame({"Demand Point":dp_cords_list})

source_cords.to_csv("source_cords.csv",index=False)
dp_cords.to_csv("dp_cords.csv",index=False)

## Calculating Distance Matrix then using speed v cal. time matrix

### Based on haversine method i.e. static (doesn't consider historical data and avg actual time on the road)

### Matrix will be 16x26 - df_psxdf_dp

In [23]:
import geopy.distance as geodesic

distance_matrix=np.zeros((len(s_cords_list),len(dp_cords_list)))

for i in range(len(s_cords_list)):
    for j in range (len(dp_cords_list)):
        distance_matrix[i,j]=geodesic.geodesic(s_cords_list[i],dp_cords_list[j]).km


In [24]:
distance_matrix

array([[36.64362222, 16.72418593,  6.95084892, 27.90407593, 30.458255  ,
        19.50906552, 35.45018067, 18.9250956 ,  1.84855651, 10.11512605,
        36.20340224, 35.34239433, 32.58125099, 36.20113396, 25.7124358 ,
        16.38184221, 11.68804769, 24.17365765, 28.019405  , 16.86928994,
         3.17266727, 37.53053616,  2.91705207, 36.6601415 , 34.87084594,
        17.14696962],
       [46.38340445, 10.03474093, 17.07885931, 30.89373485, 40.98393936,
        28.63363949, 42.11314416, 18.3121944 , 11.34219964,  0.53220338,
        44.60250681, 40.91541951, 43.20036917, 46.37544484, 26.97617354,
         7.14663462, 21.59710686, 33.56999873, 38.29244871, 13.40997256,
         7.79855977, 46.68653894, 13.51919599, 44.33970153, 45.42787943,
        26.38972174],
       [50.58793125,  4.77849116, 24.56608589, 29.4759165 , 47.95325147,
        36.67863886, 43.1040138 , 15.84417381, 17.74807585,  8.87473619,
        47.21233656, 40.99032994, 49.68764182, 51.28849786, 24.58123912,
       

In [25]:
np.savetxt('distance_matrix.csv', distance_matrix, delimiter=',')

## Converting Distance Matrix to Time Matrix v=100km/hr

In [26]:
velocity=110 # in km/hr
time_array=distance_matrix/velocity

In [27]:
np.savetxt('time_matrix.csv',time_array,delimiter=',')